# OS11 - Exercises 7-8

In [1]:
from sympy import *

In [2]:
import numpy as np

In [3]:
import scipy.linalg as linalg

In [4]:
from scipy.optimize import linprog

### Previous functions useful here

In [7]:
def nodes_indexer(named_nodes):
    nodes = {}
    for i in range(len(named_nodes)):
        nodes[named_nodes[i]] = i
    return nodes

In [83]:
def ANMatrix_from_arcs(graph, num_nodes):
    num_arcs = len(graph)
    arcs = []
    MN = np.zeros((num_nodes, num_arcs)).astype(int)
    for a in range(len(graph)):
        MN[graph[a][0][0], a] = 1
        MN[graph[a][0][1], a] = -1
        arcs.append(graph[a][0])
    return MN

## Ex. 7 - Evacuation plan 

In [128]:
named_nodes = ['s', 'a', 'b', 't']   #node enumeration. Can be any non repeated inmutable.

In [129]:
nodes = nodes_indexer(named_nodes)

In [130]:
timed_graph = []

timed_graph.append((('s', 'a'), (5,1)))   #capacidad, tiempo
timed_graph.append((('s', 'b'), (10,3)))
timed_graph.append((('a', 't'), (3,1)))
timed_graph.append((('a', 'b'), (6,2)))
timed_graph.append((('b', 't'), (3,1)))

timed_graph


[(('s', 'a'), (5, 1)),
 (('s', 'b'), (10, 3)),
 (('a', 't'), (3, 1)),
 (('a', 'b'), (6, 2)),
 (('b', 't'), (3, 1))]

### Time Expansion

I will create new time-derived nodes

In [131]:
t_horizon = 6

In [132]:
t_named_nodes = [str(n) + '_' + str(i) for n in named_nodes for i in range(t_horizon)]
t_named_nodes.append('s')
t_named_nodes.append('t')

In [133]:
t_nodes = nodes_indexer(t_named_nodes)

Now, for each arc, I will "translate on time":

In [134]:
graph = []

for arc in timed_graph:
    for t in range(t_horizon):
        t_dest = t + arc[1][1]
        if t_dest < t_horizon:
            n_origen = arc[0][0] + '_' + str(t)
            n_dest = arc[0][1] + '_' + str(t_dest)
            n_cost = arc[1][0]
            graph.append(((n_origen, n_dest), n_cost))

In [135]:
graph

[(('s_0', 'a_1'), 5),
 (('s_1', 'a_2'), 5),
 (('s_2', 'a_3'), 5),
 (('s_3', 'a_4'), 5),
 (('s_4', 'a_5'), 5),
 (('s_0', 'b_3'), 10),
 (('s_1', 'b_4'), 10),
 (('s_2', 'b_5'), 10),
 (('a_0', 't_1'), 3),
 (('a_1', 't_2'), 3),
 (('a_2', 't_3'), 3),
 (('a_3', 't_4'), 3),
 (('a_4', 't_5'), 3),
 (('a_0', 'b_2'), 6),
 (('a_1', 'b_3'), 6),
 (('a_2', 'b_4'), 6),
 (('a_3', 'b_5'), 6),
 (('b_0', 't_1'), 3),
 (('b_1', 't_2'), 3),
 (('b_2', 't_3'), 3),
 (('b_3', 't_4'), 3),
 (('b_4', 't_5'), 3)]

Graph is actually incomplete. We must add node s and t, and related arcs.

In [136]:
for t in range(t_horizon):
    graph.append((('s', 's_' + str(t)), None))
    graph.append((('t_' + str(t), 't'), None))

graph.append((('t', 's'), None))

graph

[(('s_0', 'a_1'), 5),
 (('s_1', 'a_2'), 5),
 (('s_2', 'a_3'), 5),
 (('s_3', 'a_4'), 5),
 (('s_4', 'a_5'), 5),
 (('s_0', 'b_3'), 10),
 (('s_1', 'b_4'), 10),
 (('s_2', 'b_5'), 10),
 (('a_0', 't_1'), 3),
 (('a_1', 't_2'), 3),
 (('a_2', 't_3'), 3),
 (('a_3', 't_4'), 3),
 (('a_4', 't_5'), 3),
 (('a_0', 'b_2'), 6),
 (('a_1', 'b_3'), 6),
 (('a_2', 'b_4'), 6),
 (('a_3', 'b_5'), 6),
 (('b_0', 't_1'), 3),
 (('b_1', 't_2'), 3),
 (('b_2', 't_3'), 3),
 (('b_3', 't_4'), 3),
 (('b_4', 't_5'), 3),
 (('s', 's_0'), None),
 (('t_0', 't'), None),
 (('s', 's_1'), None),
 (('t_1', 't'), None),
 (('s', 's_2'), None),
 (('t_2', 't'), None),
 (('s', 's_3'), None),
 (('t_3', 't'), None),
 (('s', 's_4'), None),
 (('t_4', 't'), None),
 (('s', 's_5'), None),
 (('t_5', 't'), None),
 (('t', 's'), None)]

Graph data is in named form, I need in indexed form:

In [137]:
indexed_graph = [((t_nodes[a[0][0]], t_nodes[a[0][1]]), a[1]) for a in graph]
indexed_graph

[((0, 7), 5),
 ((1, 8), 5),
 ((2, 9), 5),
 ((3, 10), 5),
 ((4, 11), 5),
 ((0, 15), 10),
 ((1, 16), 10),
 ((2, 17), 10),
 ((6, 19), 3),
 ((7, 20), 3),
 ((8, 21), 3),
 ((9, 22), 3),
 ((10, 23), 3),
 ((6, 14), 6),
 ((7, 15), 6),
 ((8, 16), 6),
 ((9, 17), 6),
 ((12, 19), 3),
 ((13, 20), 3),
 ((14, 21), 3),
 ((15, 22), 3),
 ((16, 23), 3),
 ((24, 0), None),
 ((18, 25), None),
 ((24, 1), None),
 ((19, 25), None),
 ((24, 2), None),
 ((20, 25), None),
 ((24, 3), None),
 ((21, 25), None),
 ((24, 4), None),
 ((22, 25), None),
 ((24, 5), None),
 ((23, 25), None),
 ((25, 24), None)]

Now, I can create the arcs-nodes matrix

In [138]:
A = ANMatrix_from_arcs(indexed_graph, len(t_nodes))
A

array([[ 1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
         0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        -1,  0,  0],
       [ 0

Cost function:

In [139]:
c = np.zeros((len(graph)))
c[-1]=-1
c

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.])

El verctor de igualdades b:
    

In [140]:
b = np.zeros((len(t_nodes)))
b

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

Límites para cada arco:

In [141]:
limites = [(0, a[1]) for a in graph]
limites

[(0, 5),
 (0, 5),
 (0, 5),
 (0, 5),
 (0, 5),
 (0, 10),
 (0, 10),
 (0, 10),
 (0, 3),
 (0, 3),
 (0, 3),
 (0, 3),
 (0, 3),
 (0, 6),
 (0, 6),
 (0, 6),
 (0, 6),
 (0, 3),
 (0, 3),
 (0, 3),
 (0, 3),
 (0, 3),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None)]

### Resolution

In [142]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, method='revised simplex')

<ipython-input-142-62916b0d23f2>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, method='revised simplex')


In [143]:
r

     con: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])
     fun: -18.0
 message: 'Optimization terminated successfully.'
     nit: 19
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 3.,  3.,  3.,  3., -0.,  3.,  3.,  0.,  0.,  3.,  3.,  3.,  3.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,  6.,  0.,  6.,  0.,
        3.,  3.,  3.,  3.,  0.,  6., -0.,  6., 18.])

### Results

Flow across the graph on t=6:

In [148]:
r.x[-1]

18.0

## Ex. 8 - Transshipment Problem